In [39]:
import nltk
import string
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [40]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import spacy
from collections import Counter

In [41]:
import pandas as pd
df = pd.read_csv('Coachella-2015-2-DFE.csv',encoding='latin-1', on_bad_lines='skip')
df


,coachella_sentiment,coachella_yn,name,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,positive,yes,kokombil,0,#Coachella2015 tickets selling out in less tha...,"[0.0, 0.0]",1/7/15 15:02,5.529630e+17,NaN,Quito
1,positive,yes,MisssTaraaa10,2,RT @sudsybuddy: WAIT THIS IS ABSOLUTE FIRE _ÙÓ...,NaN,1/7/15 15:02,5.529630e+17,united states,NaN
2,positive,yes,NMcCracken805,0,#Coachella2015 #VIP passes secured! See you th...,NaN,1/7/15 15:01,5.529630e+17,"Costa Mesa, CA",NaN
3,positive,yes,wxpnfm,1,PhillyÛªs @warondrugsjams will play #Coachell...,NaN,1/7/15 15:01,5.529630e+17,"Philadelphia, PA and Worldwide",Quito
4,positive,yes,Caesears,0,If briana and her mom out to #Coachella2015 i...,NaN,1/7/15 15:00,5.529630e+17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3841,cant tell,yes,MissXOverdose,0,Excuse me while I go cry now. _Ù÷¢ #Coachella2...,NaN,1/6/15 10:32,5.525330e+17,atx,Pacific Time (US & Canada)
3842,cant tell,yes,NedRaggett,1,RT @touchofallright: Ride and Steely Dan. It w...,NaN,1/6/15 10:28,5.525320e+17,"Costa Mesa, CA",Pacific Time (US & Canada)
3843,cant tell,yes,nicolejackieee,0,I've been callin that since day one #Coachella...,NaN,1/6/15 10:26,5.525320e+17,NaN,NaN
3844,cant tell,yes,NiqueWobbitz,2,Is this for real?!?! #Coachella2015 http://t.c...,NaN,1/6/15 10:21,5.525300e+17,Huntington Beach X Long Beach,Pacific Time (US & Canada)


In [42]:
np.sum(df.isnull().any(axis=1))

3690

In [43]:
df['coachella_sentiment'].unique()

array(['positive', 'neutral', 'negative', 'cant tell'], dtype=object)

In [44]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
df['text']= df['text'].apply(lambda x: cleaning_punctuations(x))
df['text'].tail()

3841    Excuse me while I go cry now Ù÷¢ Coachella2015...
3842    RT touchofallright Ride and Steely Dan It woul...
3843     Ive been callin that since day one Coachella2015
3844     Is this for real Coachella2015 httptcoPbeDWs5OS9
3845    RT JenBaz Cmonnnnnn lineup coachella2015 Impat...
Name: text, dtype: object

Cleaning and removing repeating characters

In [45]:
def cleaning_repeating_char(text):
    return re.sub(r'(.)1+', r'1', text)
df['text'] = df['text'].apply(lambda x: cleaning_repeating_char(x))
df['text'].tail()

3841    Excuse me while I go cry now Ù÷¢ Coachella215 ...
3842    RT touchofallright Ride and Steely Dan It woul...
3843      Ive been callin that since day one Coachella215
3844      Is this for real Coachella215 httptcoPbeDWs5OS9
3845    RT JenBaz Cmonnnnnn lineup coachella215 Impatient
Name: text, dtype: object

Cleaning and removing URLs

In [46]:
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
df['text'] = df['text'].apply(lambda x: cleaning_URLs(x))
df['text'].tail()

3841    Excuse me while I go cry now Ù÷¢ Coachella215 ...
3842    RT touchofallright Ride and Steely Dan It woul...
3843      Ive been callin that since day one Coachella215
3844      Is this for real Coachella215 httptcoPbeDWs5OS9
3845    RT JenBaz Cmonnnnnn lineup coachella215 Impatient
Name: text, dtype: object

Cleaning and removing numeric numbers

In [47]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
df['text'] =df['text'].apply(lambda x: cleaning_numbers(x))
df['text'].tail()

3841    Excuse me while I go cry now Ù÷¢ Coachella htt...
3842    RT touchofallright Ride and Steely Dan It woul...
3843         Ive been callin that since day one Coachella
3844           Is this for real Coachella httptcoPbeDWsOS
3845       RT JenBaz Cmonnnnnn lineup coachella Impatient
Name: text, dtype: object

In [48]:
df

,coachella_sentiment,coachella_yn,name,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,positive,yes,kokombil,0,Coachella tickets selling out in less than mi...,"[0.0, 0.0]",1/7/15 15:02,5.529630e+17,NaN,Quito
1,positive,yes,MisssTaraaa10,2,RT sudsybuddy WAIT THIS IS ABSOLUTE FIRE ÙÓ´ÙÓ...,NaN,1/7/15 15:02,5.529630e+17,united states,NaN
2,positive,yes,NMcCracken805,0,Coachella VIP passes secured See you there bit...,NaN,1/7/15 15:01,5.529630e+17,"Costa Mesa, CA",NaN
3,positive,yes,wxpnfm,1,PhillyÛªs warondrugsjams will play Coachella ...,NaN,1/7/15 15:01,5.529630e+17,"Philadelphia, PA and Worldwide",Quito
4,positive,yes,Caesears,0,If briana and her mom out to Coachella im out...,NaN,1/7/15 15:00,5.529630e+17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3841,cant tell,yes,MissXOverdose,0,Excuse me while I go cry now Ù÷¢ Coachella htt...,NaN,1/6/15 10:32,5.525330e+17,atx,Pacific Time (US & Canada)
3842,cant tell,yes,NedRaggett,1,RT touchofallright Ride and Steely Dan It woul...,NaN,1/6/15 10:28,5.525320e+17,"Costa Mesa, CA",Pacific Time (US & Canada)
3843,cant tell,yes,nicolejackieee,0,Ive been callin that since day one Coachella,NaN,1/6/15 10:26,5.525320e+17,NaN,NaN
3844,cant tell,yes,NiqueWobbitz,2,Is this for real Coachella httptcoPbeDWsOS,NaN,1/6/15 10:21,5.525300e+17,Huntington Beach X Long Beach,Pacific Time (US & Canada)


In [49]:
def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens
df['text'] = df['text'].apply(preprocess_text)

In [50]:
df

,coachella_sentiment,coachella_yn,name,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,positive,yes,kokombil,0,"[coachella, tickets, selling, less, minutes, ù...","[0.0, 0.0]",1/7/15 15:02,5.529630e+17,NaN,Quito
1,positive,yes,MisssTaraaa10,2,"[rt, sudsybuddy, wait, absolute, fire, ùó´ùó´ù...",NaN,1/7/15 15:02,5.529630e+17,united states,NaN
2,positive,yes,NMcCracken805,0,"[coachella, vip, passes, secured, see, bitches...",NaN,1/7/15 15:01,5.529630e+17,"Costa Mesa, CA",NaN
3,positive,yes,wxpnfm,1,"[phillyûªs, warondrugsjams, play, coachella, ...",NaN,1/7/15 15:01,5.529630e+17,"Philadelphia, PA and Worldwide",Quito
4,positive,yes,Caesears,0,"[briana, mom, coachella, im, ù÷ýù÷ýù÷ýùõä]",NaN,1/7/15 15:00,5.529630e+17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3841,cant tell,yes,MissXOverdose,0,"[excuse, go, cry, ù÷¢, coachella, httptconqdbmav]",NaN,1/6/15 10:32,5.525330e+17,atx,Pacific Time (US & Canada)
3842,cant tell,yes,NedRaggett,1,"[rt, touchofallright, ride, steely, dan, would...",NaN,1/6/15 10:28,5.525320e+17,"Costa Mesa, CA",Pacific Time (US & Canada)
3843,cant tell,yes,nicolejackieee,0,"[ive, callin, since, day, one, coachella]",NaN,1/6/15 10:26,5.525320e+17,NaN,NaN
3844,cant tell,yes,NiqueWobbitz,2,"[real, coachella, httptcopbedwsos]",NaN,1/6/15 10:21,5.525300e+17,Huntington Beach X Long Beach,Pacific Time (US & Canada)


In [51]:
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
df['text']= df['text'].apply(lambda x: stemming_on_text(x))
df['text'].head()

0    [coachella, tickets, selling, less, minutes, ù...
1    [rt, sudsybuddy, wait, absolute, fire, ùó´ùó´ù...
2    [coachella, vip, passes, secured, see, bitches...
3    [phillyûªs, warondrugsjams, play, coachella, ...
4           [briana, mom, coachella, im, ù÷ýù÷ýù÷ýùõä]
Name: text, dtype: object

In [52]:
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
df['text'] = df['text'].apply(lambda x: lemmatizer_on_text(x))
df['text'].head()

0    [coachella, tickets, selling, less, minutes, ù...
1    [rt, sudsybuddy, wait, absolute, fire, ùó´ùó´ù...
2    [coachella, vip, passes, secured, see, bitches...
3    [phillyûªs, warondrugsjams, play, coachella, ...
4           [briana, mom, coachella, im, ù÷ýù÷ýù÷ýùõä]
Name: text, dtype: object

In [53]:
X=df.text
y=df.coachella_sentiment

In [54]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05, random_state =26105111)

In [56]:
X_train_str = [' '.join(tokens) for tokens in X_train]
X_test_str = [' '.join(tokens) for tokens in X_test]

In [64]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_str)
X_test_tfidf = tfidf_vectorizer.transform(X_test_str)

In [65]:
svm_model = LinearSVC()
svm_model.fit(X_train_tfidf, y_train)

LinearSVC()

In [66]:
y_pred = svm_model.predict(X_test_tfidf)

In [67]:
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_rep)

Confusion Matrix:
[[  0   2   1   1]
 [  0   8   7   5]
 [  0   4  21  22]
 [  0   7  10 105]]

Classification Report:
              precision    recall  f1-score   support

   cant tell       0.00      0.00      0.00         4
    negative       0.38      0.40      0.39        20
     neutral       0.54      0.45      0.49        47
    positive       0.79      0.86      0.82       122

    accuracy                           0.69       193
   macro avg       0.43      0.43      0.43       193
weighted avg       0.67      0.69      0.68       193



In [72]:
sentiment_mapping = {
    0: "negative",
    1: "neutral",
    2: "positive"}

In [73]:
y_pred_sentiments = svm_model.predict(X_test_tfidf)
results_df = pd.DataFrame({'Text': X_test, 'Predicted Sentiment': y_pred_sentiments})
print(results_df.head())

                                                   Text Predicted Sentiment
3276  [rt, dopeposts, first, coachella, govballnyc, ...             neutral
2331    [wants, buy, fucking, shuttle, pass, coachella]             neutral
3763                    [coachella, fersure, thing, ï]            positive
1504  [florence, fucking, machine, actual, tears, ù÷...            positive
28    [happy, announce, nicolabott, twins, tickets, ...            positive
